<a href="https://colab.research.google.com/github/vfrantc/dehaze_detect/blob/main/run_refinednet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Fri Nov 19 11:44:04 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
!git clone https://github.com/xiaofeng94/RefineDNet-for-dehazing.git

Cloning into 'RefineDNet-for-dehazing'...
remote: Enumerating objects: 126, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 126 (delta 52), reused 103 (delta 37), pack-reused 0
Receiving objects: 100% (126/126), 2.41 MiB | 1.01 MiB/s, done.
Resolving deltas: 100% (52/52), done.


In [7]:
%cd RefineDNet-for-dehazing

/content/RefineDNet-for-dehazing


In [8]:
!cp /content/drive/MyDrive/haze/data/RTTS.zip .
!cp /content/drive/MyDrive/refined_DCP_outdoor.zip .
!unzip RTTS.zip
!unzip refined_DCP_outdoor.zip

Streaming output truncated to the last 5000 lines.
  inflating: RTTS/Annotations/SY_Baidu_141.xml  
  inflating: RTTS/Annotations/SY_Baidu_148.xml  
  inflating: RTTS/Annotations/SY_Baidu_156.xml  
  inflating: RTTS/Annotations/SY_Baidu_158.xml  
  inflating: RTTS/Annotations/SY_Baidu_160.xml  
  inflating: RTTS/Annotations/SY_Baidu_184.xml  
  inflating: RTTS/Annotations/SY_Baidu_186.xml  
  inflating: RTTS/Annotations/SY_Baidu_190.xml  
  inflating: RTTS/Annotations/SY_Baidu_196.xml  
  inflating: RTTS/Annotations/SY_Baidu_208.xml  
  inflating: RTTS/Annotations/SY_Baidu_227.xml  
  inflating: RTTS/Annotations/SY_Baidu_25.xml  
  inflating: RTTS/Annotations/SY_Baidu_262.xml  
  inflating: RTTS/Annotations/SY_Baidu_292.xml  
  inflating: RTTS/Annotations/SY_Baidu_295.xml  
  inflating: RTTS/Annotations/SY_Baidu_337.xml  
  inflating: RTTS/Annotations/SY_Baidu_346.xml  
  inflating: RTTS/Annotations/SY_Baidu_367.xml  
  inflating: RTTS/Annotations/SY_Baidu_371.xml  
  inflating: RTTS/A

In [29]:
import os
import cv2
from glob import glob
from tqdm.notebook import tqdm

In [ ]:
!mkdir checkpoints
!mv refined_DCP_outdoor checkpoints
!mkdir datasets/rtts
!cp -r RTTS/JPEGImages datasets/rtts/testA

In [31]:
sizes = dict()
for imfile in tqdm(glob('datasets/rtts/testA/*.png')):
  image = cv2.imread(imfile)
  (height, width) = image.shape[:2]
  key = os.path.basename(imfile)
  sizes[key] = (height, width)

  height, width = image.shape[:2]
  msize = min(height, width)
  if msize < 256:
    if height < width:
      ratio = width / height
      new_height = 256
      new_width = int(new_height*ratio)
    else:
      ratio = height / width
      new_width = 256
      new_height = int(new_width*ratio)
    image = cv2.resize(image, (new_width, new_height))
    cv2.imwrite(imfile, image)

  0%|          | 0/4322 [00:00<?, ?it/s]

In [32]:
!python3 quick_test.py --dataroot ./datasets/rtts --dataset_mode single --name refined_DCP_outdoor --model refined_DCP --phase test --preprocess none --save_image --method_name refined_DCP_outdoor_ep_60 --epoch 60

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: ./datasets/rtts               	[default: ./datasets/ITS_v2]
             dataset_mode: single                        	[default: unaligned]
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: 60                            	[default: latest]
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
    

In [33]:
sizes = dict()
for imfile in tqdm(glob('out_refined/JPEGImages/*.png')):
  image = cv2.imread(imfile)
  (height, width) = image.shape[:2]
  key = os.path.basename(imfile)
  if (sizes[key][0] != height) or (sizes[key][1] != width):
    image = cv2.resize(image, (sizes[key][1], sizes[key][0]))
    cv2.imwrite(imfile, image)

0it [00:00, ?it/s]

In [37]:
!rm -rf out_refined
!mkdir out_refined
!cp /content/drive/MyDrive/new_test.json out_refined
!mv datasets/rtts/refined_DCP_outdoor_ep_60 out_refined/JPEGImages
!tar czf out_refined.tar.gz out_refined/
!cp out_refined.tar.gz /content/drive/MyDrive/hazy_detect/

In [38]:
!ls

checkpoints  options		 README.md		  test_BeDDE.py
data	     out_refined	 refined_DCP_outdoor.zip  train.py
datasets     out_refined.tar.gz  RTTS			  util
models	     quick_test.py	 RTTS.zip


In [39]:
!ls

checkpoints  options		 README.md		  test_BeDDE.py
data	     out_refined	 refined_DCP_outdoor.zip  train.py
datasets     out_refined.tar.gz  RTTS			  util
models	     quick_test.py	 RTTS.zip
